In [3]:
import requests
import pandas as pd

In [ ]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
standings_url

In [ ]:
data = requests.get(standings_url)

In [ ]:
data.text

In [4]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(data.text)

In [ ]:
standings_table = soup.select('table.stats_table')[0]
# standings_table = soup.select("#results2024-202591_overall")
# test = pd.read_html(data.text, match = "Premier League")
# test
standings_table

In [ ]:
links = standings_table.find_all('a')
# links = soup.select('a')

In [ ]:
links = [l.get("href") for l in links]

In [ ]:
links = [l for l in links if '/squads/' in l]

In [ ]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [ ]:
team_urls

In [ ]:
team_url = team_urls[0]

In [ ]:
data = requests.get(team_url)

In [ ]:
matches = pd.read_html(data.text, match = "Scores & Fixtures")
matches[0].head()

In [ ]:
soup = BeautifulSoup(data.text)

In [ ]:
links = soup.find_all('a')

In [ ]:
links = [l.get("href") for l in links]
links

In [ ]:
links = [l for l in links if l and "all_comps/shooting/" in l]

In [ ]:
links

In [ ]:
data = requests.get(f"https://fbref.com{links[0]}")

In [ ]:
shooting = pd.read_html(data.text, match = "Shooting")[0]

In [ ]:
shooting.columns = shooting.columns.droplevel()

In [ ]:
shooting.head()

In [ ]:
team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on = "Date")

In [ ]:
team_data.head()

In [6]:
years = list(range(2018, 2013, -1))

In [7]:
years

[2018, 2017, 2016, 2015, 2014]

In [8]:
all_matches = []

In [9]:
# standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

standings_url = "https://fbref.com/en/comps/9/2018-2019/2018-2019-Premier-League-Stats"

In [ ]:
import time
for year in years:
    data = requests.get(standings_url)
    time.sleep(3)
    text = data.text
    time.sleep(3)
    soup = BeautifulSoup(text)
    time.sleep(3)
    standings_table = soup.select('table.stats_table')[0]
    time.sleep(3)
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    time.sleep(3)
    links = [l.get("href") for l in standings_table.find_all('a')]
    time.sleep(3)
    links = [l for l in links if '/squads/' in l]
    time.sleep(3)
    team_urls = [f"https://fbref.com{l}" for l in links]
    time.sleep(3)

    
    for team_url in team_urls:
        time.sleep(3)
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        time.sleep(3)
        # print(team_name) # testing
        time.sleep(3)

        data = requests.get(team_url)
        time.sleep(3)
        text = data.text
        time.sleep(3)
        matches = pd.read_html(data.text, match = "Scores & Fixtures")[0]
        time.sleep(3)
        print(f"\n\n{year}: First 5 matches for {team_name}: {matches.head()}\n") #testing
        time.sleep(3)

        soup = BeautifulSoup(text)
        time.sleep(3)
        links = [l.get("href") for l in soup.find_all('a')]
        time.sleep(3)
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        time.sleep(3)
        data = requests.get(f"https://fbref.com{links[0]}")
        time.sleep(3)
        text = data.text
        time.sleep(3)
        # print(data.text)
        shooting = pd.read_html(text, match = "Shooting")[0]
        time.sleep(3)
        shooting.columns = shooting.columns.droplevel()
        time.sleep(3)

        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on = "Date")
            time.sleep(3)
        except ValueError:
            continue

        team_data = team_data[team_data["Comp"] == "Premier League"]
        time.sleep(3)
        team_data["Season"] = year
        time.sleep(3)
        team_data["Team"] = team_name
        time.sleep(3)
        all_matches.append(team_data)
        time.sleep(3)

In [12]:
print(all_matches)

[          Date   Time            Comp         Round  Day Venue Result GF GA  \
1   2018-08-12  16:00  Premier League   Matchweek 1  Sun  Away      W  2  0   
2   2018-08-19  13:30  Premier League   Matchweek 2  Sun  Home      W  6  1   
3   2018-08-25  12:30  Premier League   Matchweek 3  Sat  Away      D  1  1   
4   2018-09-01  17:30  Premier League   Matchweek 4  Sat  Home      W  2  1   
5   2018-09-15  15:00  Premier League   Matchweek 5  Sat  Home      W  3  0   
7   2018-09-22  15:00  Premier League   Matchweek 6  Sat  Away      W  5  0   
9   2018-09-29  15:00  Premier League   Matchweek 7  Sat  Home      W  2  0   
11  2018-10-07  16:30  Premier League   Matchweek 8  Sun  Away      D  0  0   
12  2018-10-20  15:00  Premier League   Matchweek 9  Sat  Home      W  5  0   
14  2018-10-29  20:00  Premier League  Matchweek 10  Mon  Away      W  1  0   
16  2018-11-04  15:00  Premier League  Matchweek 11  Sun  Home      W  6  1   
18  2018-11-11  16:30  Premier League  Matchweek 12

In [14]:
match_df2 = pd.concat(all_matches)
match_df2.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
1,2018-08-12,16:00,Premier League,Matchweek 1,Sun,Away,W,2,0,Arsenal,...,Match Report,NaN,17.0,8.0,18.2,1.0,0,0,2018,Manchester City
2,2018-08-19,13:30,Premier League,Matchweek 2,Sun,Home,W,6,1,Huddersfield,...,Match Report,NaN,32.0,14.0,18.0,1.0,0,0,2018,Manchester City
3,2018-08-25,12:30,Premier League,Matchweek 3,Sat,Away,D,1,1,Wolves,...,Match Report,NaN,18.0,6.0,18.6,1.0,0,0,2018,Manchester City
4,2018-09-01,17:30,Premier League,Matchweek 4,Sat,Home,W,2,1,Newcastle Utd,...,Match Report,NaN,24.0,8.0,17.5,3.0,0,0,2018,Manchester City
5,2018-09-15,15:00,Premier League,Matchweek 5,Sat,Home,W,3,0,Fulham,...,Match Report,NaN,28.0,9.0,16.6,1.0,0,0,2018,Manchester City


In [16]:
print(match_df2.shape)

(1520, 28)


In [17]:
match_df2.columns = [c.lower() for c in match_df2.columns]
match_df2.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2018-08-12,16:00,Premier League,Matchweek 1,Sun,Away,W,2,0,Arsenal,...,Match Report,NaN,17.0,8.0,18.2,1.0,0,0,2018,Manchester City
2,2018-08-19,13:30,Premier League,Matchweek 2,Sun,Home,W,6,1,Huddersfield,...,Match Report,NaN,32.0,14.0,18.0,1.0,0,0,2018,Manchester City
3,2018-08-25,12:30,Premier League,Matchweek 3,Sat,Away,D,1,1,Wolves,...,Match Report,NaN,18.0,6.0,18.6,1.0,0,0,2018,Manchester City
4,2018-09-01,17:30,Premier League,Matchweek 4,Sat,Home,W,2,1,Newcastle Utd,...,Match Report,NaN,24.0,8.0,17.5,3.0,0,0,2018,Manchester City
5,2018-09-15,15:00,Premier League,Matchweek 5,Sat,Home,W,3,0,Fulham,...,Match Report,NaN,28.0,9.0,16.6,1.0,0,0,2018,Manchester City


In [18]:
match_df2.to_csv("oldermatches.csv")

In [46]:
match_df = pd.concat(all_matches)
match_df.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
1,2018-08-12,16:00,Premier League,Matchweek 1,Sun,Away,W,2,0,Arsenal,...,Match Report,NaN,17.0,8.0,18.2,1.0,0,0,2018,Manchester City
2,2018-08-19,13:30,Premier League,Matchweek 2,Sun,Home,W,6,1,Huddersfield,...,Match Report,NaN,32.0,14.0,18.0,1.0,0,0,2018,Manchester City
3,2018-08-25,12:30,Premier League,Matchweek 3,Sat,Away,D,1,1,Wolves,...,Match Report,NaN,18.0,6.0,18.6,1.0,0,0,2018,Manchester City
4,2018-09-01,17:30,Premier League,Matchweek 4,Sat,Home,W,2,1,Newcastle Utd,...,Match Report,NaN,24.0,8.0,17.5,3.0,0,0,2018,Manchester City
5,2018-09-15,15:00,Premier League,Matchweek 5,Sat,Home,W,3,0,Fulham,...,Match Report,NaN,28.0,9.0,16.6,1.0,0,0,2018,Manchester City


In [47]:
match_df.columns = [c.lower() for c in match_df.columns]

In [48]:
match_df.to_csv("matches2.csv")

In [20]:
df_old = pd.read_csv("oldermatches.csv")
df_old.head()

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,1,2018-08-12,16:00,Premier League,Matchweek 1,Sun,Away,W,2,0,...,Match Report,NaN,17.0,8.0,18.2,1.0,0,0,2018,Manchester City
1,2,2018-08-19,13:30,Premier League,Matchweek 2,Sun,Home,W,6,1,...,Match Report,NaN,32.0,14.0,18.0,1.0,0,0,2018,Manchester City
2,3,2018-08-25,12:30,Premier League,Matchweek 3,Sat,Away,D,1,1,...,Match Report,NaN,18.0,6.0,18.6,1.0,0,0,2018,Manchester City
3,4,2018-09-01,17:30,Premier League,Matchweek 4,Sat,Home,W,2,1,...,Match Report,NaN,24.0,8.0,17.5,3.0,0,0,2018,Manchester City
4,5,2018-09-15,15:00,Premier League,Matchweek 5,Sat,Home,W,3,0,...,Match Report,NaN,28.0,9.0,16.6,1.0,0,0,2018,Manchester City


In [21]:
df_new = pd.read_csv("matches.csv")

df_new.head()

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,0,2024-08-17,12:30,Premier League,Matchweek 1,Sat,Away,W,2.0,0.0,...,Match Report,NaN,18.0,5.0,14.8,0.0,0,0,2024,Liverpool
1,1,2024-08-25,16:30,Premier League,Matchweek 2,Sun,Home,W,2.0,0.0,...,Match Report,NaN,19.0,8.0,13.6,1.0,0,0,2024,Liverpool
2,2,2024-09-01,16:00,Premier League,Matchweek 3,Sun,Away,W,3.0,0.0,...,Match Report,NaN,11.0,3.0,13.4,0.0,0,0,2024,Liverpool
3,3,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,L,0.0,1.0,...,Match Report,NaN,14.0,5.0,14.9,0.0,0,0,2024,Liverpool
4,5,2024-09-21,15:00,Premier League,Matchweek 5,Sat,Home,W,3.0,0.0,...,Match Report,NaN,19.0,12.0,16.6,0.0,0,0,2024,Liverpool


In [22]:
combined_df = pd.concat([df_new, df_old], ignore_index = True)


In [31]:
print(f"New match shape: {df_new.shape}")
print(f"Old match shape: {df_old.shape}")
print(f"Total rows for appended df should be: {df_new.shape[0] + df_old.shape[0]}")

New match shape: (4258, 29)
Old match shape: (1520, 29)
Total rows for appended df should be: 5778


In [23]:
combined_df.shape # should be 5778

(5778, 29)

In [38]:
combined_df.rename(columns = {"Unnamed: 0" : "MatchID"}, inplace = True)

In [43]:
index_range = range(combined_df.shape[0])

In [44]:
combined_df["MatchID"] = index_range

In [45]:
combined_df.to_csv("combined_file.csv", index = False)